In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

#from torchvision import datasets
#from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

import pandas as pd

#https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

https://medium.com/@shashikachamod4u/excel-csv-to-pytorch-dataset-def496b6bcc1

In [3]:
torch.manual_seed(0)

csv = pd.read_csv("class77.csv")
csv = shuffle(csv)

pre_full = csv
pre_train=csv[0:2500]
pre_test=csv[2500:5000]

In [4]:
class csvDataset(Dataset):
    def __init__(self,file):
        file_out = file #pd.read_csv(file)
        x=file_out.iloc[:,1:6].values
        y=file_out.iloc[:,6:7].values

        sc = StandardScaler()
        x_train = sc.fit_transform(x)
        y_train = y
        
        self.X_train = torch.tensor(x_train,dtype = torch.float32)
        self.y_train = torch.tensor(y_train)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self,idx):
        return self.X_train[idx].float(),self.y_train[idx].float()
    
        

In [5]:
full = csvDataset(pre_full)
train = csvDataset(pre_train)
test = csvDataset(pre_test)

In [6]:
batch_size =500
full_dataloader = DataLoader(full, batch_size=batch_size)
train_dataloader = DataLoader(train, batch_size=batch_size)
test_dataloader = DataLoader(test, batch_size=batch_size)


If you want to load a model use this

In [7]:

#model = NeuralNetwork()
#model.load_state_dict(torch.load("model.pth"))

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [8]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 11),
            nn.ReLU(),
            nn.Linear(11, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=11, bias=True)
    (1): ReLU()
    (2): Linear(in_features=11, out_features=2, bias=True)
    (3): ReLU()
  )
)


Here we build the neural network itself. 
We use the https://en.wikipedia.org/wiki/Universal_approximation_theorem
to get a rough idea of how big of a network we need.

There are two versions of this : an older one from the 50s that deals only 
with shallow networks of arbitrary width ("shallow neural nets").
And a newer one that deals with networks of arbitrary depth (e.g. deep neural nets). 

By using the UAT, I hope to answer the question: 
Am I in the money or am I completely in the left field of potatoes even before I reach the starting line? 

Because I want to respect the https://en.wikipedia.org/wiki/KISS_principle,
so instead of starting right away with, 

In this case, the theorem says: the magic number is . 
In practice, it might be a little bit less, a little bit more, but we'll go with "a little bit less" in this case. 

I'm going to try the same architecture for two cases:
One for class 77, the "hard" class, and one for class 16, the "easy" class.


In [9]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 12),
            nn.ReLU(),
            nn.Linear(12, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=12, bias=True)
    (3): ReLU()
    (4): Linear(in_features=12, out_features=2, bias=True)
    (5): ReLU()
  )
)


https://pytorch.org/docs/stable/nn.html#loss-functions


https://pytorch.org/docs/stable/optim.html

https://ruder.io/optimizing-gradient-descent/index.html#gradientdescentvariants

https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

https://analyticsindiamag.com/ultimate-guide-to-pytorch-optimizers/

In [10]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False) #torch.optim.SGD(model.parameters(), lr=1e-1)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print(size)
    num_batches = len(dataloader)
    #model.eval() will notify all your layers that you are in eval mode
    model.eval()
    test_loss, correct = 0, 0
    #torch.no_grad() impacts the autograd engine and deactivate it. It will reduce memory usage and speed up computations but you won’t be able to backprop
    with torch.no_grad():
        for X, y in dataloader:
            #print(f"this is y{y}")
            X, y = X.to(device), y.to(device)
            pred = model(X)
            #print(f"this is pred:{pred}")
            test_loss += loss_fn(pred, y).item()
            
            #print(f"this is predargmax{pred.argmax(1)}")
            
            final_pred = pred.argmax(1)            

            
            #correct += (pred.argmax(1) == y.flatten()).type(torch.float).sum().item()
            #print(torch.eq(final_pred,y.flatten()))
            correct += torch.eq(final_pred,y.flatten()).sum().item()

            #print(f"this is condition{(pred.argmax(1) == y)}")
            #print(f"This is correct:{correct}")
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e

In [13]:
#for g in optimizer.param_groups:
#    print(g)

In [19]:
epochs = 2000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.001532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.7%, Avg loss: 0.002398 

Epoch 2
-------------------------------
loss: 0.001532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 6.0%, Avg loss: 0.002398 

Epoch 3
-------------------------------
loss: 0.001535  [    0/ 2500]
2500
Test Error: 
 Accuracy: 1.2%, Avg loss: 0.002414 

Epoch 4
-------------------------------
loss: 0.001527  [    0/ 2500]
2500
Test Error: 
 Accuracy: 1.1%, Avg loss: 0.002457 

Epoch 5
-------------------------------
loss: 0.001533  [    0/ 2500]
2500
Test Error: 
 Accuracy: 37.2%, Avg loss: 0.002476 

Epoch 6
-------------------------------
loss: 0.001537  [    0/ 2500]
2500
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.002456 

Epoch 7
-------------------------------
loss: 0.001519  [    0/ 2500]
2500
Test Error: 
 Accuracy: 4.0%, Avg loss: 0.002455 

Epoch 8
-------------------------------
loss: 0.001521  [    0/ 2500]
2500
Test Error: 
 Accuracy: 58.4%, Avg loss:

In [18]:
#If you want to save the model. 
#torch.save(model.state_dict(), "Vanilla SGD model.pth")
#print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


So as you can see our experiment ends in failure: 
it yoyos all over the place

The loss itself is not 